#### 1. Importing necessary libraries

In [3]:
import kagglehub


#### 2. Extracting raw data

100%|██████████| 959k/959k [00:00<00:00, 1.57MB/s]

Extracting files...
Path to dataset files: C:\Users\Jam\.cache\kagglehub\datasets\patelris\crop-yield-prediction-dataset\versions\1


#### 3. Transform data into consistent formats
- Column names are consistent and formatted
- Normalization of data to organize entities

#### 4. Loading transformed data into PostgreSQL

#### 5. Query and analyze data
- Use mathplot and seaborn for analysis
- Use PandasProfiling for full packaged analysis


#### 6. Apply machine learning